In [2]:
class_names = ["床面", "椅子", "収納", "玄関", "浴槽", "洗面台"]

class_labels = { i: class_name for i, class_name in enumerate(class_names)}
class_ids = [i for i in range(len(class_names))]


def get_class_key(class_name):
    return class_names.index(class_name)

def get_class_name(class_key):
    return class_names[class_key]

In [3]:
import json

annotation_dir = "dataset/annotations/"

In [4]:
from keras_cv import bounding_box
import tensorflow as tf
import numpy as np


def load_annotation(annotation_file):
    annotations = []

    with open(annotation_file) as f:
        annotation_dict = json.load(f)

    for k, data in annotation_dict.items():
        file_name = data["filename"]
        size = data["size"]
        regions = data["regions"]
        bounding_boxes = []

        for region in regions:
            bbox = convert_bbox_to_kerascv_format(region)

            if bbox is None:
                continue

            bounding_boxes.append(bbox)

        annotations.append({
            "file_name": file_name,
            "size": size,
            "bounding_boxes": bounding_boxes
        })

    return annotations

def convert_bbox_to_kerascv_format(region):
    shape_attributes = region['shape_attributes']
    region_attributes = region['region_attributes']

    shape_name = shape_attributes['name']

    if shape_name != 'rect':
        print('shape name is not rect', shape_name)
        return None

    x = shape_attributes['x']
    y = shape_attributes['y']
    width = shape_attributes['width']
    height = shape_attributes['height']
    class_name = region_attributes['class']

    boxes = {
        'classes': get_class_key(class_name),
        "boxes": np.array([x, y, x + width, y + height])
    }
    bbox = bounding_box.convert_format(boxes, 'xywh', 'xywh')

    return bbox

def load_image(image_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, (512, 512))
    return image

2024-05-02 08:50:25.897064: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-02 08:50:25.934941: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-02 08:50:26.494857: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/onikarubi/documents/team-shiny-products/floor-plan-ocr/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .aut

In [5]:
annotation_dict = load_annotation(
    annotation_file=annotation_dir + "via_project_15Apr2024_9h49m_json.json"
)

annotation_dict

shape name is not rect polyline
shape name is not rect polyline


2024-05-02 08:50:31.152381: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-02 08:50:31.158467: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


[{'file_name': '1005310.jpg',
  'size': 11046,
  'bounding_boxes': [{'classes': 0,
    'boxes': <tf.Tensor: shape=(4,), dtype=float32, numpy=array([ 14., 243., 104., 267.], dtype=float32)>},
   {'classes': 2,
    'boxes': <tf.Tensor: shape=(4,), dtype=float32, numpy=array([ 15.,  91.,  55., 117.], dtype=float32)>},
   {'classes': 3,
    'boxes': <tf.Tensor: shape=(4,), dtype=float32, numpy=array([60., 29., 94., 40.], dtype=float32)>},
   {'classes': 2,
    'boxes': <tf.Tensor: shape=(4,), dtype=float32, numpy=array([43., 29., 59., 42.], dtype=float32)>},
   {'classes': 4,
    'boxes': <tf.Tensor: shape=(4,), dtype=float32, numpy=array([14., 44., 55., 86.], dtype=float32)>},
   {'classes': 0,
    'boxes': <tf.Tensor: shape=(4,), dtype=float32, numpy=array([16., 29., 44., 42.], dtype=float32)>},
   {'classes': 5,
    'boxes': <tf.Tensor: shape=(4,), dtype=float32, numpy=array([ 86.,  47., 101.,  63.], dtype=float32)>}]}]

In [ ]:
region = {
    "shape_attributes": {"name": "rect", "x": 15, "y": 91, "width": 40, "height": 26},
    "region_attributes": {
        "floor_label": "収納",
        "class": "収納",
        "description": "収納スペース",
    },
}

bbox = convert_bbox_to_kerascv_format(region)
print("Converted BoundingBox:", bbox)

In [ ]:
from PIL import Image, ImageDraw, ImageFont

def draw_bbox(image_path, bbox, color='red', width=3):
    image = Image.open(image_path)
    draw = ImageDraw.Draw(image)

    boxes = bbox['boxes']
    classes = bbox['classes']

    draw.rectangle(boxes, outline=color, width=width)

    label_position = (boxes[0], boxes[1] - 18)
    draw.text(label_position, get_class_name(classes), fill=color)
    image.show()

img_path = "dataset/Set_A_02/color/1005310.jpg"
draw_bbox(img_path, bbox)